In [1]:


import tensorflow as tf
from IPython.core.display import Image
from keras._tf_keras.keras.preprocessing import image
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils, layers
import numpy as np
import os




In [2]:
# Підготовка даних (як у умові)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test  = x_test.reshape(10000, 784).astype('float32') / 255.0

y_train = utils.to_categorical(y_train, 10)
y_test  = utils.to_categorical(y_test, 10)

print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)


x_train shape: (60000, 784) y_train shape: (60000, 10)


### Модель з додатковим прихованим шаром

In [3]:
def build_model_part2(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=128, max_value=1024, step=32),
                    input_dim=784,
                    activation='relu'))
    model.add(Dense(units=hp.Int('units_hidden', min_value=64, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


### Тюнер — BayesianOptimization

In [4]:
from kerastuner.tuners import BayesianOptimization

tuner = BayesianOptimization(
    build_model_part2,
    objective='val_accuracy',
    max_trials=8,
    executions_per_trial=1,
    directory='tuner_part2_dir',
    project_name='part2_bayes'
)

tuner.search_space_summary()



Search space summary
Default search space size: 2
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


C:\Users\dasav\AppData\Local\Temp\ipykernel_16772\1791485016.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization
C:\Users\dasav\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
tuner.search(x_train,                  # Дані для навчання
             y_train,                  # Вірні відповіді
             batch_size=256,           # Розмір міні-вибірки
             epochs=20,                # Кількість епох навчання
             validation_split=0.2,     # Частина даних, яка буде використовуватися для перевірки
             )

Trial 8 Complete [00h 03m 50s]
val_accuracy: 0.8479999899864197

Best val_accuracy So Far: 0.8544166684150696
Total elapsed time: 00h 29m 47s


In [6]:
tuner.results_summary()

Results summary
Results in tuner_part2_dir\part2_bayes
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 5 summary
Hyperparameters:
units_input: 832
units_hidden: 96
Score: 0.8544166684150696

Trial 4 summary
Hyperparameters:
units_input: 832
units_hidden: 64
Score: 0.8540833592414856

Trial 0 summary
Hyperparameters:
units_input: 512
units_hidden: 384
Score: 0.8538333177566528

Trial 2 summary
Hyperparameters:
units_input: 640
units_hidden: 256
Score: 0.8527500033378601

Trial 6 summary
Hyperparameters:
units_input: 512
units_hidden: 352
Score: 0.8499166369438171

Trial 7 summary
Hyperparameters:
units_input: 800
units_hidden: 96
Score: 0.8479999899864197

Trial 3 summary
Hyperparameters:
units_input: 512
units_hidden: 416
Score: 0.8476666808128357

Trial 1 summary
Hyperparameters:
units_input: 192
units_hidden: 256
Score: 0.8416666388511658


In [7]:
models = tuner.get_best_models(num_models=3)
for model in models:
    model.summary()
    model.evaluate(x_test, y_test)
    print()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 832)            │       653,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │        79,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 734,058 (2.80 MB)

 Trainable params: 734,058 (2.80 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │       196,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         3,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 602,762 (2.30 MB)

 Trainable params: 602,762 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8434 - loss: 0.4502



In [10]:
top_model = tuner.get_best_models(num_models=1)
top_model[0].save('models/task2-best.keras')
Image('shirt.jpg', width=150, height=150)
img = image.load_img('shirt.jpg', target_size=(28, 28), color_mode="grayscale")
# Перетворюмо картинку в масив
x = image.img_to_array(img)
# Змінюємо форму масиву в плаский вектор
x = x.reshape(1, 784)
# Інвертуємо зображення
x = 255 - x
# Нормалізуємо зображення
x /= 255

In [9]:
classes = ['футболка', 'брюки', 'светр', 'плаття', 'пальто', 'туфлі', 'сорочка', 'кросівки', 'сумка', 'чоботи']
prediction = top_model[0].predict(x)
prediction = np.argmax(prediction)
print("Номер класу:", prediction)
print("Назва класу:", classes[prediction])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
Номер класу: 6
Назва класу: сорочка
